In [38]:
import pandas as pd
import twint
import nest_asyncio
nest_asyncio.apply()
import re
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import sklearn.metrics as metrics
import re
import string
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from datetime import datetime, timedelta
import pandas_datareader.data as pdr
import datetime as dt

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\prade\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Data Collection with TWINT

In [2]:
# Initialize twint
c = twint.Config()

In [3]:
# Search
# Performed search for both A Pompliano,and Coin bureau for BTC from 2020-10-06 to 2021-10-06

cbbit = 'Bitcoin (from:coinbureau) until:2021-10-06 since:2020-10-06'
c = twint.Config()
c.Search = cbbit
c.Store_csv = True
c.Output = "cbbit.csv"

# Run and save to CSV
twint.run.Search(c)

# Data was collected in CSV file which were concacted with pandas df method, and resaved in csv file

1443962973152858112 2021-10-01 10:36:14 -0500 <coinbureau> It's the start of a new Quarter, and already #Bitcoin is flying! Not surprising when you consider the institutional demand we are seeing. In my video today, I look at a recent survey that had me feeling quite bullish!  P.S. Enjoy the disclaimer 🏴‍☠️   https://t.co/bUsNDR3jiw
1443268329062469635 2021-09-29 12:35:58 -0500 <coinbureau> Bitcoin tips is not the only exciting thing Twitter has been working on 👀  #NFTs
1442481437366759429 2021-09-27 08:29:09 -0500 <coinbureau> Fun Fact: $1 invested in Gold 10 years ago, is worth 95 cents today 🤯  A dollar hidden under your pillow would have served as a better "store of value" and "inflation hedge"  #Bitcoin  https://t.co/Nu0Iymd3Tz
1441368478775013378 2021-09-24 06:46:39 -0500 <coinbureau> China FUD™: Bringing you volatile trading conditions since 2017!  #Bitcoin
1440646137828970506 2021-09-22 06:56:19 -0500 <coinbureau> When in doubt, zoom out...  #Bitcoin
1436329897283227661 2021-09

### Sentiment anlaysis, and price action for Anthony Pompliano @APompliano

In [4]:
# Re read combined csv
pomp_tweets = pd.read_csv('pompbtc.csv')

In [5]:
pomp_tweets.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1445517131177545738,1445517131177545738,2021-10-05 17:31:55 Central Daylight Time,2021-10-05,17:31:55,-500,339061487,apompliano,Pomp 🌪,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1445486015532072964,1445486015532072964,2021-10-05 15:28:16 Central Daylight Time,2021-10-05,15:28:16,-500,339061487,apompliano,Pomp 🌪,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1445403387680460801,1445403387680460801,2021-10-05 09:59:56 Central Daylight Time,2021-10-05,09:59:56,-500,339061487,apompliano,Pomp 🌪,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1445369324110295043,1445369324110295043,2021-10-05 07:44:35 Central Daylight Time,2021-10-05,07:44:35,-500,339061487,apompliano,Pomp 🌪,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1445362244565487620,1445362244565487620,2021-10-05 07:16:27 Central Daylight Time,2021-10-05,07:16:27,-500,339061487,apompliano,Pomp 🌪,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
# Dropping unnecessary columns

pomp_tweets = pomp_tweets.drop(['id', 'conversation_id', 'created_at', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'language',
       'urls', 'photos','link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'], axis = 1)



In [7]:
pomp_tweets.head()

,date,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags
0,2021-10-05,The SEC has just said they will not ban Bitcoi...,[],103,278,2078,[],[]
1,2021-10-05,BREAKING: SEC Chairman Gary Gensler says the S...,[],1002,4483,28691,[],[]
2,2021-10-05,Today's episode of The Best Business Show is ...,"[{'screen_name': 'peterdiamandis', 'name': 'pe...",44,32,190,[],[]
3,2021-10-05,"Bitcoin is closer to $100,000 than it is to $0.",[],496,662,8404,[],[]
4,2021-10-05,New episode with @RaoulGMI is now live! - mac...,"[{'screen_name': 'raoulgmi', 'name': 'raoul pa...",47,50,403,[],[]


In [8]:
# Index changed to date
pomp_tweets.set_index('date',inplace = True)
pomp_tweets.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags
date,,,,,,,
2021-10-05,The SEC has just said they will not ban Bitcoi...,[],103,278,2078,[],[]
2021-10-05,BREAKING: SEC Chairman Gary Gensler says the S...,[],1002,4483,28691,[],[]
2021-10-05,Today's episode of The Best Business Show is ...,"[{'screen_name': 'peterdiamandis', 'name': 'pe...",44,32,190,[],[]
2021-10-05,"Bitcoin is closer to $100,000 than it is to $0.",[],496,662,8404,[],[]
2021-10-05,New episode with @RaoulGMI is now live! - mac...,"[{'screen_name': 'raoulgmi', 'name': 'raoul pa...",47,50,403,[],[]


### Obtaining the sentiment score 

In [9]:
# Create single function to build Sentiment Analysis Compound Score column:
def compound_sentiment_columns(text):
    # Function to clean text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    # Function to get compound sentiments
    sentiment = analyzer.polarity_scores(re_clean)
    compound = sentiment["compound"]
    return compound

In [10]:
# Calculating sentiment and adding compound sentiment column
pomp_tweets['compound_sentiment'] = pomp_tweets['tweet'].apply(lambda x:compound_sentiment_columns(x))
pomp_tweets.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment
date,,,,,,,,
2021-10-05,The SEC has just said they will not ban Bitcoi...,[],103,278,2078,[],[],0.6931
2021-10-05,BREAKING: SEC Chairman Gary Gensler says the S...,[],1002,4483,28691,[],[],-0.7003
2021-10-05,Today's episode of The Best Business Show is ...,"[{'screen_name': 'peterdiamandis', 'name': 'pe...",44,32,190,[],[],0.7783
2021-10-05,"Bitcoin is closer to $100,000 than it is to $0.",[],496,662,8404,[],[],0.0000
2021-10-05,New episode with @RaoulGMI is now live! - mac...,"[{'screen_name': 'raoulgmi', 'name': 'raoul pa...",47,50,403,[],[],0.7579


In [11]:
# Index of DataFrame changed to date
pomp_tweets.index = pd.to_datetime(pomp_tweets.index)
pomp_tweets.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment
date,,,,,,,,
2021-10-05,The SEC has just said they will not ban Bitcoi...,[],103,278,2078,[],[],0.6931
2021-10-05,BREAKING: SEC Chairman Gary Gensler says the S...,[],1002,4483,28691,[],[],-0.7003
2021-10-05,Today's episode of The Best Business Show is ...,"[{'screen_name': 'peterdiamandis', 'name': 'pe...",44,32,190,[],[],0.7783
2021-10-05,"Bitcoin is closer to $100,000 than it is to $0.",[],496,662,8404,[],[],0.0000
2021-10-05,New episode with @RaoulGMI is now live! - mac...,"[{'screen_name': 'raoulgmi', 'name': 'raoul pa...",47,50,403,[],[],0.7579


## Obtaining BTC price from yahoo

In [12]:
import pandas_datareader.data as pdr
import datetime as dt

In [13]:
start = dt.datetime(2020,10,6)
end = dt.datetime(2021,10,6)
btc_df = pdr.DataReader('BTC-USD','yahoo',start,end)
btc_df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-10-05,10803.456055,10565.197266,10799.778320,10621.664062,2.226496e+10,10621.664062
2020-10-06,10687.268555,10591.963867,10619.803711,10679.136719,1.763439e+10,10679.136719
2020-10-07,10939.799805,10569.823242,10677.625000,10923.627930,2.196212e+10,10923.627930
2020-10-09,11442.210938,11056.940430,11059.142578,11296.361328,2.287798e+10,11296.361328
2020-10-10,11428.813477,11288.627930,11296.082031,11384.181641,1.996863e+10,11384.181641


In [14]:
# Dropping all columns except for CLose
btc_df = btc_df.drop(['High','Low','Open','Volume','Adj Close'], axis=1)
btc_df.head()

,Close
Date,
2020-10-05,10621.664062
2020-10-06,10679.136719
2020-10-07,10923.627930
2020-10-09,11296.361328
2020-10-10,11384.181641


In [15]:
# Calculating the daily return
btc_df['returns'] = btc_df['Close'].pct_change()
btc_df.head()

,Close,returns
Date,,
2020-10-05,10621.664062,NaN
2020-10-06,10679.136719,0.005411
2020-10-07,10923.627930,0.022894
2020-10-09,11296.361328,0.034122
2020-10-10,11384.181641,0.007774


In [16]:
# Combining sentiment analysis, and returns in single dataframe
pomp_sentiment_price = pomp_tweets.join(btc_df, how = 'outer')
pomp_sentiment_price.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment,Close,returns
2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10621.664062,NaN
2020-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10679.136719,0.005411
2020-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10923.627930,0.022894
2020-10-09,"At this point, you’re violating your fiduciary...",[],89.0,250.0,1398.0,[],[],-0.4416,11296.361328,0.034122
2020-10-09,Bitcoin waking up and reminding everyone why i...,[],148.0,254.0,2764.0,[],[],0.0000,11296.361328,0.034122


In [17]:
# Dropping null values
pomp_sentiment_price = pomp_sentiment_price.dropna()
pomp_sentiment_price.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment,Close,returns
2020-10-09,"At this point, you’re violating your fiduciary...",[],89.0,250.0,1398.0,[],[],-0.4416,11296.361328,0.034122
2020-10-09,Bitcoin waking up and reminding everyone why i...,[],148.0,254.0,2764.0,[],[],0.0000,11296.361328,0.034122
2020-10-10,63% of bitcoin's circulating supply has not be...,"[{'screen_name': 'yassineark', 'name': 'yassin...",59.0,128.0,1006.0,[],[],0.6731,11384.181641,0.007774
2020-10-13,I have no clue why @BlockFi is running a promo...,"[{'screen_name': 'blockfi', 'name': 'blockfi',...",80.0,48.0,300.0,[],[],0.3182,11429.506836,0.003981
2020-10-13,@RampCapitalLLC You were asking me how to buy ...,[],2.0,1.0,47.0,[],[],0.0000,11429.506836,0.003981


## Assigning X feature, and y target variable

In [18]:
X = pomp_sentiment_price[['replies_count', 'retweets_count', 'likes_count','compound_sentiment']].values
X

array([[ 8.900e+01,  2.500e+02,  1.398e+03, -4.416e-01],
       [ 1.480e+02,  2.540e+02,  2.764e+03,  0.000e+00],
       [ 5.900e+01,  1.280e+02,  1.006e+03,  6.731e-01],
       ...,
       [ 4.400e+01,  3.200e+01,  1.900e+02,  7.783e-01],
       [ 4.960e+02,  6.620e+02,  8.404e+03,  0.000e+00],
       [ 4.700e+01,  5.000e+01,  4.030e+02,  7.579e-01]])

In [19]:
y = pomp_sentiment_price['returns'].values
y

array([0.03412176, 0.03412176, 0.00777421, ..., 0.0746705 , 0.0746705 ,
       0.0746705 ])

In [20]:
# Splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [21]:
#  Define the pipeline for scaling and model fitting
linear_model = Pipeline([
    ("MinMax Scaling", MinMaxScaler()),  
    ("Linear Regression", LinearRegression())
])

sgd_model = Pipeline([
    ("MinMax Scaling", MinMaxScaler()),  
    ("SGD Regression", SGDRegressor())
])


In [22]:
# Using fit function to transform, and fitting the model. 
linear_model.fit(X_train, y_train)
sgd_model.fit(X_train, y_train)

Pipeline(steps=[('MinMax Scaling', MinMaxScaler()),
                ('SGD Regression', SGDRegressor())])

In [23]:
# Predicting target
y_pred1 = linear_model.predict(X_test)
y_pred2 = sgd_model.predict(X_test)

In [24]:
# Checking score and absolute error

print('Pipeline 1 Mean Absolute Error for Linear Regression: ', mean_absolute_error(y_pred1, y_test))
print('Pipeline 1 Score for Linear Regression', linear_model.score(X_test, y_test))

print('Pipeline 2 Mean Absolute Error for SGD Regression: ', mean_absolute_error(y_pred2, y_test))
print('Pipeline 2 Score for SGD Regression', sgd_model.score(X_test, y_test))

Pipeline 1 Mean Absolute Error for Linear Regression:  0.03347777384729437
Pipeline 1 Score for Linear Regression 0.0014997224602659731
Pipeline 2 Mean Absolute Error for SGD Regression:  0.033693758537129144
Pipeline 2 Score for SGD Regression -0.005339434929698683


### Sentiment anlaysis, and price action for twitter handle @CoinBureau

In [25]:
# Re read combined csv
coinbureau_tweets = pd.read_csv('coinbureaubtc.csv')

In [26]:
coinbureau_tweets.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1394684985357991937,1394684985357991937,2021-05-18 11:03:07 Central Daylight Time,2021-05-18,11:03:07,-500,906230721513181184,coinbureau,Coin Bureau (guy.eth),NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1394324644887400455,1394324644887400455,2021-05-17 11:11:15 Central Daylight Time,2021-05-17,11:11:15,-500,906230721513181184,coinbureau,Coin Bureau (guy.eth),NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1394239635749015554,1394239635749015554,2021-05-17 05:33:27 Central Daylight Time,2021-05-17,05:33:27,-500,906230721513181184,coinbureau,Coin Bureau (guy.eth),NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1393958983958814726,1393958983958814726,2021-05-16 10:58:14 Central Daylight Time,2021-05-16,10:58:14,-500,906230721513181184,coinbureau,Coin Bureau (guy.eth),NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1393853568797577216,1393853568797577216,2021-05-16 03:59:22 Central Daylight Time,2021-05-16,03:59:22,-500,906230721513181184,coinbureau,Coin Bureau (guy.eth),NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [27]:
# Dropping unnecessary columns

coinbureau_tweets = coinbureau_tweets.drop(['id', 'conversation_id', 'created_at', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'language',
       'urls', 'photos','link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'], axis = 1)


In [28]:
coinbureau_tweets.head()

,date,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags
0,2021-05-18,"#Litecoin is the OG altcoin. The ""silver"" to B...",[],72,93,885,['litecoin'],['ltc']
1,2021-05-17,Coin Bureau Weekly News is live! We have a lot...,[],22,47,468,['bitcoin'],[]
2,2021-05-17,Looking forward to the Coin Bureau Weekly news...,[],173,75,2461,['bitcoin'],['ada']
3,2021-05-16,#Bitcoin has become a global asset that is dri...,[],28,50,554,['bitcoin'],[]
4,2021-05-16,Most people overestimate their #crypto gains i...,[],51,77,808,"['crypto', 'bitcoin']",[]


In [29]:
# Index changed to date
coinbureau_tweets.set_index('date',inplace = True)
coinbureau_tweets.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags
date,,,,,,,
2021-05-18,"#Litecoin is the OG altcoin. The ""silver"" to B...",[],72,93,885,['litecoin'],['ltc']
2021-05-17,Coin Bureau Weekly News is live! We have a lot...,[],22,47,468,['bitcoin'],[]
2021-05-17,Looking forward to the Coin Bureau Weekly news...,[],173,75,2461,['bitcoin'],['ada']
2021-05-16,#Bitcoin has become a global asset that is dri...,[],28,50,554,['bitcoin'],[]
2021-05-16,Most people overestimate their #crypto gains i...,[],51,77,808,"['crypto', 'bitcoin']",[]


## Obtaining the sentiment score

In [30]:
# Create single function to build Sentiment Analysis Compound Score column:
def compound_sentiment_columns(text):
    # Function to clean text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    # Function to get compound sentiments
    sentiment = analyzer.polarity_scores(re_clean)
    compound = sentiment["compound"]
    return compound

In [31]:
# Calculating sentiment and adding compound sentiment column
coinbureau_tweets['compound_sentiment'] = coinbureau_tweets['tweet'].apply(lambda x:compound_sentiment_columns(x))
coinbureau_tweets.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment
date,,,,,,,,
2021-05-18,"#Litecoin is the OG altcoin. The ""silver"" to B...",[],72,93,885,['litecoin'],['ltc'],-0.3612
2021-05-17,Coin Bureau Weekly News is live! We have a lot...,[],22,47,468,['bitcoin'],[],0.5052
2021-05-17,Looking forward to the Coin Bureau Weekly news...,[],173,75,2461,['bitcoin'],['ada'],0.0000
2021-05-16,#Bitcoin has become a global asset that is dri...,[],28,50,554,['bitcoin'],[],0.6124
2021-05-16,Most people overestimate their #crypto gains i...,[],51,77,808,"['crypto', 'bitcoin']",[],-0.7391


In [32]:
# Index of DataFrame changed to date
coinbureau_tweets.index = pd.to_datetime(coinbureau_tweets.index)
coinbureau_tweets.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment
date,,,,,,,,
2021-05-18,"#Litecoin is the OG altcoin. The ""silver"" to B...",[],72,93,885,['litecoin'],['ltc'],-0.3612
2021-05-17,Coin Bureau Weekly News is live! We have a lot...,[],22,47,468,['bitcoin'],[],0.5052
2021-05-17,Looking forward to the Coin Bureau Weekly news...,[],173,75,2461,['bitcoin'],['ada'],0.0000
2021-05-16,#Bitcoin has become a global asset that is dri...,[],28,50,554,['bitcoin'],[],0.6124
2021-05-16,Most people overestimate their #crypto gains i...,[],51,77,808,"['crypto', 'bitcoin']",[],-0.7391


In [33]:
# Combining sentiment analysis, and returns in single dataframe
coinbureau_sentiment_price = coinbureau_tweets.join(btc_df, how = 'outer')
coinbureau_sentiment_price.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment,Close,returns
2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10621.664062,NaN
2020-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10679.136719,0.005411
2020-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10923.627930,0.022894
2020-10-09,This is some pretty exciting research for #Mon...,[],1.0,7.0,22.0,"['monero', 'bitcoin']","['xmr', 'btc']",0.7506,11296.361328,0.034122
2020-10-09,Only $50m into #Bitcoin Square? You can do bet...,[],3.0,1.0,29.0,['bitcoin'],[],0.4404,11296.361328,0.034122


In [34]:
# Dropping null values
coinbureau_sentiment_price = coinbureau_sentiment_price.dropna()
coinbureau_sentiment_price.head()

,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,cashtags,compound_sentiment,Close,returns
2020-10-09,This is some pretty exciting research for #Mon...,[],1.0,7.0,22.0,"['monero', 'bitcoin']","['xmr', 'btc']",0.7506,11296.361328,0.034122
2020-10-09,Only $50m into #Bitcoin Square? You can do bet...,[],3.0,1.0,29.0,['bitcoin'],[],0.4404,11296.361328,0.034122
2020-10-10,Another tired cliche being propagated by the D...,[],4.0,4.0,34.0,"['crypto', 'bitcoin']",[],-0.1280,11384.181641,0.007774
2020-10-10,Stimulus cheques could be around the corner. H...,[],2.0,4.0,38.0,"['stimulus', 'bitcoin']",[],0.6249,11384.181641,0.007774
2020-10-13,"Damn, another one. They just had to find a #bi...",[],1.0,1.0,6.0,['bitcoin'],[],-0.4019,11429.506836,0.003981


In [40]:
coinbureau_sentiment_price.shape

(542, 10)

## Assigning X feature, and y target variable

In [41]:
X = coinbureau_sentiment_price[['replies_count', 'retweets_count', 'likes_count','compound_sentiment']].values
X.shape

(542, 4)

In [42]:
y = coinbureau_sentiment_price['returns'].values
y.shape

(542,)

In [43]:
# Splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)